In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%204%20-%20S%2BP/S%2BP%20Week%202%20Lesson%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
import io

print(tf.__version__)

2.9.0


In [3]:
def rm_tree(pth):
    pth = Path(pth)
    for child in pth.iterdir():
        if child.is_file():
            child.unlink()
        else:
            rm_tree(child)
    pth.rmdir()


if (logdir := Path.joinpath(Path.cwd(), Path("logs"))).exists():
    rm_tree(str(logdir.resolve()))

Path.mkdir(logdir, parents=True, exist_ok=True)

# Clear out prior logging data.
if (plotdir := Path.joinpath(logdir, "plots")).exists():
    rm_tree(str(plotdir.resolve()))

plotdir = Path.joinpath(plotdir, f"{datetime.now().strftime('%Y%m%d-%H%M%S')}")
file_writer = tf.summary.create_file_writer(str(plotdir.resolve()))


def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image


def plot_series(time, series, format="-", start=0, end=None, series_name=""):
    figure = plt.figure(figsize=(10, 10))
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)
    with file_writer.as_default():
        tf.summary.image(series_name, plot_to_image(figure), step=0)


def plot_series(
    time,
    series1,
    series2,
    format="-",
    start1=0,
    end1=None,
    start2=0,
    end2=None,
    series_name="",
):
    figure = plt.figure(figsize=(10, 10))
    plt.plot(time[start1:end1], series1[start1:end1], format)
    plt.plot(time[start2:end2], series2[start2:end2], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)
    with file_writer.as_default():
        tf.summary.image(series_name, plot_to_image(figure), step=0)


def trend(time, slope=0):
    return slope * time


def seasonal_pattern(season_time):
    """Just an arbitrary pattern, you can change it if you wish"""
    return np.where(
        season_time < 0.4, np.cos(season_time * 2 * np.pi), 1 / np.exp(3 * season_time)
    )


def seasonality(time, period, amplitude=1, phase=0):
    """Repeats the same pattern at each period"""
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)


def noise(time, noise_level=1, seed=None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

In [4]:
time = np.arange(4 * 365 + 1, dtype="float32")
baseline = 10
series = trend(time, 0.1)
baseline = 10
amplitude = 40
slope = 0.05
noise_level = 5

# Create the series
series = (
    baseline + trend(time, slope) + seasonality(time, period=365, amplitude=amplitude)
)
# Update with noise
series += noise(time, noise_level, seed=42)

split_time = 1000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

window_size = 20
batch_size = 32
shuffle_buffer_size = 1000

In [10]:
def windowed_dataset(
    series,
    window_size,
    batch_size=32,
    shuffle_buffer=None,
    window_shift=1,
):
    def create_target(target_window):
        X = target_window[:-1]
        Y = target_window[-1]
        return X, Y

    def batch_window(batched_window, size):
        return batched_window.batch(size)

    ds = tf.data.Dataset.from_tensor_slices(series)
    length_dataset = (
        shuffle_buffer
        if shuffle_buffer
        else ds.reduce(0, lambda x, _: x + 1).numpy() * 50 // 100
    )
    ds = ds.window(window_size + 1, shift=window_shift, drop_remainder=True)
    ds = ds.flat_map(lambda window: batch_window(window, window_size + 1))

    ds = ds.shuffle(length_dataset)
    ds = ds.map(create_target, num_parallel_calls=tf.data.AUTOTUNE)

    ds = ds.batch(batch_size, num_parallel_calls=tf.data.AUTOTUNE)

    ds = ds.prefetch(1)
    for X_batch, Y_batch in ds.take(1):
        print(X_batch.shape, Y_batch.shape)
    return ds

In [11]:
dataset = windowed_dataset(
    x_train, window_size, batch_size, shuffle_buffer=shuffle_buffer_size
)
l0 = tf.keras.layers.Dense(1, input_shape=[window_size])
model = tf.keras.models.Sequential([l0])


model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(lr=1e-6, momentum=0.9))
model.fit(dataset, epochs=100, verbose=0)

(32, 20) (32,)


d:\codebase\.venv\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [12]:
forecast = [
    model.predict(series[time : time + window_size][np.newaxis])
    for time in range(len(series) - window_size)
]

1/1 [==============================] - 0s 16ms/step


In [13]:
forecast = forecast[split_time - window_size :]
results = np.array(forecast)[:, 0, 0]


plt.figure(figsize=(10, 6))

plot_series(time_valid, x_valid, results, series_name="forecast_results")
results.shape

(461,)

<Figure size 1000x600 with 0 Axes>

In [14]:
tf.keras.metrics.mean_absolute_error(x_valid, results).numpy()

4.9892917